# Summary

This Notebook demonstrates how the BERT model can analyze sentiment using reviews gathered from 'YELP' for a restaurant located in Stuttgart, Germany.

# Contents

1. Imports
2. BERT Model Initialisation
3. Playing with the Model
4. Collect the reviews from yelp
5. Score the reviews

# 1. Imports

In [1]:
import re

import pandas as pd
import requests
import torch
from bs4 import BeautifulSoup
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# 2. Model Initialisation

In [2]:
tokenizer = AutoTokenizer.from_pretrained(
    "nlptown/bert-base-multilingual-uncased-sentiment"
)

model = AutoModelForSequenceClassification.from_pretrained(
    "nlptown/bert-base-multilingual-uncased-sentiment"
)

# 3. Play With the model

In [3]:
tokens = tokenizer.encode(
    "Horrible experience, cockroaches found in the food. But it was a sweet :)",
    return_tensors="pt",
)

In [4]:
# Check how the tokens are generated
tokens

tensor([[  101, 36129, 45795, 10301, 16277,   117, 88764, 10167, 57796, 17919,
         11973, 10104, 10103, 15225,   119, 10502, 10197, 10140,   143, 19355,
           131,   114,   102]])

In [5]:
# How to decode the tokens
tokenizer.decode(tokens[0])

'[CLS] horrible experience, cockroaches found in the food. but it was a sweet : ) [SEP]'

In [6]:
# Predict the sentiment of the text
result = model(tokens)
print(result)

SequenceClassifierOutput(loss=None, logits=tensor([[ 2.0687,  1.0235, -0.2574, -1.1230, -1.4550]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


In [7]:
# The output from the model is a one hot encoded vector of size 5. Higher the value in a list that is the sentiment of the text. In this case 1- Worst, 5 - Best
result.logits

tensor([[ 2.0687,  1.0235, -0.2574, -1.1230, -1.4550]],
       grad_fn=<AddmmBackward0>)

In [8]:
int(torch.argmax(result.logits)) + 1

1

# 4. Collect the reviews from Yelp

If we navigate to the restaurant's webpage and perform a right-click on the comments section, then select 'Inspect', we'll notice that the comments begin with 'comment__'. Therefore, it's logical to use this pattern to filter the comments

In [9]:
r = requests.get("https://www.yelp.com/biz/block-house-eberhardstra%C3%9Fe-stuttgart-2")

In [11]:
restaurant_page_html = BeautifulSoup(r.text, "html.parser")

In [12]:
regex = re.compile(".*comment.*")

In [13]:
results = restaurant_page_html.find_all("p", {"class": regex})
reviews = [result.text for result in results]

In [14]:
reviews

['Much like Jeff W. said, this place is a good American-style restaurant and is a solid option for some meat and potatoes. I had the "New York Cheeseburger" today, and while it is not quite an American-style cheeseburger (it is open-faced, not a full hamburger bun), it was very good. They use garlic bread as the platform for the burger patty and the combination is delicious. The fries are your standard French fries. I like the feel of the interior of this place. It\'s like an elegant tavern, lots of wood and very cozy. Overall a good experience, I will probably check this place out again.',
 'Friendly and cozy environment with American country music. Ordered the prawns as an appetizer and was very surprised at how well the dish was combined and seasoned. It was light, refreshing, and flavorful. Served with bread almost crustini-like which was a nice touch. Main course was a ribeye with a baked potato and bread side. The ribeye was cooked medium rare as requested. But not with the prope

# 5. Score the reviews

In [15]:
reviews_df = pd.DataFrame(reviews, columns=["review"])

In [16]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors="pt")
    result = model(tokens)
    return int(torch.argmax(result.logits)) + 1

In [17]:
reviews_df["sentiment"] = reviews_df["review"].apply(lambda x: sentiment_score(x[:512]))

In [18]:
reviews_df

,review,sentiment
0,"Much like Jeff W. said, this place is a good A...",4
1,Friendly and cozy environment with American co...,4
2,My first visit to a German steak house. This r...,2
3,"Dear Karen, you are right that this is our sta...",5
4,Delicious steak cook just right. We were seati...,5
5,I saw a couple of these throughout Germany and...,4
6,Very nice meal. Very good service but very fas...,4
7,"Typical German ""American Steak House""; definit...",3
8,Don't understand the hype of this place. Decor...,2
9,"OMG!!! Am I really in Stuttgart, Germany? The ...",5
